**1. Research and data analysis**

Make a colab notebook

In [11]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Upload data
from google.colab import files
uploaded = files.upload()

# Load the Excel sheet
df = pd.read_excel('ADHD Symptom Self-Assessment (Responses).xlsx')
print("Data shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())

Saving ADHD Symptom Self-Assessment (Responses).xlsx to ADHD Symptom Self-Assessment (Responses) (1).xlsx
Data shape: (43, 39)

First 5 rows:
                Timestamp  Gender         Age  \
0 2025-10-01 11:20:23.213  Female  18 + years   
1 2025-10-08 18:16:05.921    Male  18 + years   
2 2025-10-08 19:45:47.259    Male  18 + years   
3 2025-10-08 20:46:52.054    Male  18 + years   
4 2025-10-08 21:14:16.312  Female  18 + years   

  Do you avoid starting tasks when they seem long or difficult?    \
0                                             Rarely                
1                                             Always                
2                                              Often                
3                                              Never                
4                                          Sometimes                

  When you need to focus, do you often end up doing something else instead?    \
0                                          Sometimes               

Check data

In [12]:
# Details of data
print("Columns:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)

# Check the having 'Level ' row
if 'Level' in df.columns:
    print("\nLevel distribution:")
    print(df['Level'].value_counts())
else:
    print("'Level' column not found. Checking last column...")
    print(df.iloc[:, -1].value_counts())

Columns: ['Timestamp', 'Gender', 'Age', 'Do you avoid starting tasks when they seem long or difficult?  ', 'When you need to focus, do you often end up doing something else instead?  ', 'Do you lose attention quickly when working on academic or difficult tasks?  ', 'Do you skip important tasks and spend more time on less important ones?  ', 'Do you often forget to complete important tasks or assignments? ', 'Do you frequently misplace or lose personal items, like keys, phone, or documents? ', 'Do you have trouble remembering appointments, deadlines, or schedules? ', 'Do you often forget instructions or details given by others? ', 'Do you find it difficult to organize tasks, materials, or work effectively? ', 'Are your workspace, study area, or personal belongings often messy or cluttered? ', 'Do you struggle to plan ahead and manage your time efficiently? ', 'Do you often forget steps or lose track of progress in multi-step tasks? ', 'Do you find it hard to stay focused on one task for

**2. Data pre-processing and model training**

Cleaning data

In [13]:
# Remove unecessary rows
columns_to_drop = ['Timestamp', 'Email', 'Break', 'Marks']
df_clean = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# Get the final row as a target
target_column = df_clean.columns[-1]
print(f"Target column: {target_column}")

# Assign numerical values to responses
response_mapping = {
    'Never': 0,
    'Rarely': 1,
    'Sometimes': 2,
    'Often': 3,
    'Always': 4
}

# Encode the row of the questions
question_columns = df_clean.columns[2:-1]  # පළමු columns 2 අතහරින්න (Gender, Age)
for col in question_columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].map(response_mapping)

# Encode age and gender
le_gender = LabelEncoder()
df_clean['Gender'] = le_gender.fit_transform(df_clean['Gender'])

le_age = LabelEncoder()
df_clean['Age'] = le_age.fit_transform(df_clean['Age'])

# Encode the target
le_target = LabelEncoder()
df_clean[target_column] = le_target.fit_transform(df_clean[target_column])

print("Data after preprocessing:")
print(df_clean.head())

Target column: Level
Data after preprocessing:
   Gender  Age  \
0       0    1   
1       1    1   
2       1    1   
3       1    1   
4       0    1   

   Do you avoid starting tasks when they seem long or difficult?    \
0                                                  1                 
1                                                  4                 
2                                                  3                 
3                                                  0                 
4                                                  2                 

   When you need to focus, do you often end up doing something else instead?    \
0                                                  2                             
1                                                  4                             
2                                                  4                             
3                                                  2                             
4            

Train the model

In [16]:
# Separate features and target
X = df_clean.drop(columns=[target_column])
y = df_clean[target_column]

# Split data without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test performance
y_pred = model.predict(X_test)

# Convert the numerical classes from le_target.classes_ to strings for display purposes
string_class_names = [str(cls) for cls in le_target.classes_]

print("Classification Report:")
print(classification_report(y_test, y_pred, labels=sorted(y.unique()), target_names=string_class_names))


Classification Report:
                          precision    recall  f1-score   support

        High ADHD Level        0.00      0.00      0.00         1
         Low ADHD Level        0.00      0.00      0.00         1
    Moderate ADHD Level        0.78      1.00      0.88         7
abhimaniwiraji@gmail.com       0.00      0.00      0.00         0

                accuracy                           0.78         9
               macro avg       0.19      0.25      0.22         9
            weighted avg       0.60      0.78      0.68         9



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

Save The model

In [18]:
# ආකෘතිය සුරකින්න
with open('adhd_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Encoders සුරකින්න
encoders = {
    'gender_encoder': le_gender,
    'age_encoder': le_age,
    'target_encoder': le_target,
    'response_mapping': response_mapping
}

with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

print("Model and encoders saved successfully!")

# කේතය download කරන්න
from google.colab import files

# Streamlit app කේතය
app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# පිටු සැකසුම
st.set_page_config(page_title="ADHD Assessment Tool", layout="wide")

# Sidebar
st.sidebar.title("🧭 සංචාලනය")
page = st.sidebar.radio(
    "පිටුව තෝරන්න:",
    ["🏠 මුල් පිටුව", "📊 දත්ත විශ්ලේෂණය", "📈 දෘශ්‍යකරණය", "🔮 ADHD මට්ටම් පුරෝකථනය", "📋 ආකෘති කාර්ය සාධනය"]
)

# දත්ත load කිරීම
@st.cache_data
def load_data():
    df = pd.read_excel('ADHD Symptom Self-Assessment (Responses).xlsx')
    return df

df = load_data()

# ආකෘතිය සහ encoders load කිරීම
@st.cache_resource
def load_model():
    with open('adhd_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('encoders.pkl', 'rb') as f:
        encoders = pickle.load(f)
    return model, encoders

model, encoders = load_model()

# පිටු 1: මුල් පිටුව
if page == "🏠 මුල් පිටුව":
    st.title("🎯 ADHD ස්වයං-ඇගයීමේ මෙවලම")
    st.markdown("---")

    st.subheader("සාදරයෙන් පිළිගනිමු!")
    st.write("""
    මෙම ඇප් එක ADHD රෝග ලක්ෂණ ඇගයීම සඳහා නිර්මාණය කර ඇත.
    පහත දැක්වෙන අංශ භාවිතා කර ඔබට:
    """)

    col1, col2, col3 = st.columns(3)

    with col1:
        st.markdown("### 📊 දත්ත විශ්ලේෂණය")
        st.write("දත්ත පෙරදසුන සහ සංඛ්‍යාන")

    with col2:
        st.markdown("### 📈 දෘශ්‍යකරණය")
        st.write("ග්‍රැෆ් සහ චාට් භාවිතයෙන් දත්ත දෘශ්‍යකරණය")

    with col3:
        st.markdown("### 🔮 පුරෝකථනය")
        st.write("ADHD මට්ටම් පුරෝකථනය කිරීම")

    st.markdown("---")
    st.info("💡 **සටහන:** මෙය ස්වයං-ඇගයීමේ මෙවලමක් පමණි. නිශ්චිත විද්‍යාත්මක නිර්ණය සඳහා වෛද්‍යවරයකු සමඟ පරීක්ෂා කරන්න.")

# පිටු 2: දත්ත විශ්ලේෂණය
elif page == "📊 දත්ත විශ්ලේෂණය":
    st.title("📊 දත්ත විශ්ලේෂණය")

    tab1, tab2, tab3 = st.tabs(["දත්ත පෙරදසුන", "සංඛ්‍යාන", "පෙරහන් කිරීම"])

    with tab1:
        st.subheader("දත්ත පෙරදසුන")
        st.dataframe(df.head(10))

        st.subheader("දත්ත හැඩය")
        st.write(f"පේළි: **{df.shape[0]}**, තීරු: **{df.shape[1]}**")

    with tab2:
        st.subheader("මූලික සංඛ්‍යාන")
        st.write(df.describe())

        st.subheader("දත්ත වර්ග")
        st.write(df.dtypes)

    with tab3:
        st.subheader("දත්ත පෙරහන් කිරීම")
        filter_col = st.selectbox("පෙරහන් කිරීමට තීරුව තෝරන්න", df.columns)

        if df[filter_col].dtype == 'object':
            unique_vals = df[filter_col].unique()
            selected_val = st.selectbox("තේරීම", unique_vals)
            filtered_df = df[df[filter_col] == selected_val]
        else:
            min_val = float(df[filter_col].min())
            max_val = float(df[filter_col].max())
            selected_range = st.slider("පරාසය තෝරන්න", min_val, max_val, (min_val, max_val))
            filtered_df = df[(df[filter_col] >= selected_range[0]) & (df[filter_col] <= selected_range[1])]

        st.dataframe(filtered_df)
        st.write(f"පෙරහන් කළ පේළි: **{len(filtered_df)}**")

# පිටු 3: දෘශ්‍යකරණය
elif page == "📈 දෘශ්‍යකරණය":
    st.title("📈 දත්ත දෘශ්‍යකරණය")

    chart_type = st.selectbox(
        "චාට් වර්ගය තෝරන්න",
        ["තීරු රූපය", "පිරිපැතුරුම් රූපය", "විසිරුම් රූපය", "පෙට්ටි රූපය"]
    )

    if chart_type == "තීරු රූපය":
        col = st.selectbox("තීරුව තෝරන්න", df.select_dtypes(include=['object']).columns)
        fig, ax = plt.subplots(figsize=(10, 6))
        df[col].value_counts().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_title(f'{col} - තීරු රූපය')
        ax.set_xlabel(col)
        ax.set_ylabel('ගණන')
        st.pyplot(fig)

    elif chart_type == "පිරිපැතුරුම් රූපය":
        col = st.selectbox("තීරුව තෝරන්න", df.select_dtypes(include=[np.number]).columns)
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.hist(df[col].dropna(), bins=20, color='lightgreen', edgecolor='black')
        ax.set_title(f'{col} - පිරිපැතුරුම් රූපය')
        ax.set_xlabel(col)
        ax.set_ylabel('සංඛ්‍යාතය')
        st.pyplot(fig)

# පිටු 4: පුරෝකථනය
elif page == "🔮 ADHD මට්ටම් පුරෝකථනය":
    st.title("🔮 ADHD මට්ටම් පුරෝකථනය")

    st.write("කරුණාකර පහත ප්‍රශ්නවලට පිළිතුරු සපයන්න:")

    with st.form("prediction_form"):
        col1, col2 = st.columns(2)

        with col1:
            gender = st.selectbox("ලිංගභේදය", df['Gender'].unique())
            age = st.selectbox("වයස් කාණ්ඩය", df['Age'].unique())

        with col2:
            # සාම්පල් ප්‍රශ්න කිහිපයක්
            q1 = st.selectbox("දුෂ්කර කාර්යයන් ආරම්භ කිරීමෙන් වලකිනවාද?",
                             ["Never", "Rarely", "Sometimes", "Often", "Always"])
            q2 = st.selectbox("අවධානය යොමු කිරීමට අවශ්‍ය වූ විට වෙනත් දේවල් කරනවාද?",
                             ["Never", "Rarely", "Sometimes", "Often", "Always"])

        submit = st.form_submit_button("පුරෝකථනය කරන්න")

    if submit:
        with st.spinner("පුරෝකථනය කරමින්..."):
            # ආදාන දත්ත සකස් කිරීම
            input_data = {
                'Gender': encoders['gender_encoder'].transform([gender])[0],
                'Age': encoders['age_encoder'].transform([age])[0],
                'Q1': encoders['response_mapping'][q1],
                'Q2': encoders['response_mapping'][q2]
            }

            # අනෙකුත් ප්‍රශ්න සඳහා සාමාන්‍ය අගයන්
            for i in range(3, 33):  # ඉතිරි ප්‍රශ්න
                input_data[f'Q{i}'] = 2  # 'Sometimes' ලෙස default

            # DataFrame එකක් සාදන්න
            input_df = pd.DataFrame([input_data])

            # පුරෝකථනය
            prediction = model.predict(input_df)
            prediction_label = encoders['target_encoder'].inverse_transform(prediction)[0]

            # ප්‍රතිඵල පෙන්වන්න
            st.success("✅ පුරෝකථනය සම්පූර්ණයි!")

            col1, col2, col3 = st.columns(3)

            with col1:
                st.metric("ලිංගභේදය", gender)
            with col2:
                st.metric("වයස් කාණ්ඩය", age)
            with col3:
                st.metric("පුරෝකථනය කළ ADHD මට්ටම", prediction_label)

            # විස්තර
            st.info(f"""
            **විස්තර:** {prediction_label}

            **සටහන:** මෙය කෘතීම බුද්ධිය මත පදනම් වූ පුරෝකථනයක් පමණි.
            නිශ්චිත විද්‍යාත්මක නිර්ණය සඳහා වෛද්‍යවරයකු සමඟ පරීක්ෂා කරන්න.
            """)

# පිටු 5: ආකෘති කාර්ය සාධනය
elif page == "📋 ආකෘති කාර්ය සාධනය":
    st.title("📋 ආකෘති කාර්ය සාධනය")

    # පුහුණු දත්ත යළි ලබාගන්න
    X = df_clean.drop(columns=[target_column])
    y = df_clean[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    tab1, tab2 = st.tabs(["වර්ගීකරණ වාර්තාව", "ව්‍යාකූලතා අනුකෘතිය"])

    with tab1:
        st.subheader("වර්ගීකරණ වාර්තාව")
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=encoders['target_encoder'].classes_, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        st.dataframe(report_df.style.format("{:.2f}"))

    with tab2:
        st.subheader("ව්‍යාකූලතා අනුකෘතිය")
        cm = confusion_matrix(y_test, y_pred)
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=encoders['target_encoder'].classes_,
                    yticklabels=encoders['target_encoder'].classes_,
                    ax=ax)
        ax.set_xlabel('පුරෝකථනය')
        ax.set_ylabel('සත්‍ය')
        ax.set_title('ව්‍යාකූලතා අනුකෘතිය')
        st.pyplot(fig)

# පාදක තොරතුරු
st.sidebar.markdown("---")
st.sidebar.info(
    """
    **📁 Project Files:**
    - adhd_app.py (Streamlit app)
    - adhd_model.pkl (Trained model)
    - encoders.pkl (Encoders)
    - ADHD_Symptom_Self_Assessment.xlsx (Dataset)
    """
)
'''

# කේතය .py ගොනුවක් ලෙස සුරකින්න
with open('adhd_app.py', 'w') as f:
    f.write(app_code)

# Download කරන්න
files.download('adhd_app.py')

Model and encoders saved successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>